In [1]:
# Jan 30, 2020
# Joined the 5 classification tables available
# to be evaluated:
    # join is left by default
    # because I adopt cit_received as the leftmost data
        # this dataset keep data on the patents who received citation
    # I can make the first join as an 'outer' operation between cit_received and cit_made
    # To be complete, I can begin with patent.csv.gz and make the first two joins with 'outer' type
#     Ill git add now and try this
    

# Jan 29, 2020
# many errors appeared after joining the dataset 
# i rewritten code into simpler scripts
# this script joins variables together to be processed later

# they are:
#     - cit_delay - reads clean_patent and clean_uspatentcitation and calculates delay by patent
#     - cit_made - reads clean_uspatentcitation and calculates citations made 
#     - cit_received - reads clean_uspatentcitation and calculates citations received
#     - cit_tree - reads clean_uspatentcitation and cit_made and calculates parent_citation
#     - generalit - reads wipo and clean_uspatentcitation and calculates generality
#     - originality - reads wipo and clean_uspatentcitation and calculates originality
#     - wipo_first_class - reads wipo and generates wipo_first_class

# it is now way faster, but I cannot know for sure if this is a consequence of coding or something changed in 
# infrastructure. 

# since class is a variable that also appear in non-cited patents, I'll include them in the final step


# Jan, 20th 2020
# join delay and tree data to form the analysis dataset


# In[2]:

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.delayed import delayed


def data_read(file, names, usecols, dtype):
    df = dd.read_parquet(file, compression='gzip', names=names, usecols=usecols, dtype=dtype, index_col='id', header=0)
    dfs.append(df)

In [3]:
dst='data/dataset.parquet.gz'

patent='data/cleanpatent.parquet.gz'
received='data/cit_received.parquet.gz'
made='data/cit_made.parquet.gz'
received_delay='data/cit_received_delay.parquet.gz'
made_delay='data/cit_made_delay.parquet.gz'
# cit_tree = 'data/cit_tree.csv.gz'
originality = 'data/originality.parquet.gz'
generality = 'data/generality.parquet.gz'
wipo = 'data/wipo.parquet.gz'
centrality='data/centrality_pagerank.csv'
int_ext_cit='data/int_ext_cit.parquet.gz'

In [4]:
dfs=[]

#adopting patent.csv as the reference of patents
df = dd.read_parquet(patent, compression='gzip', dtype=object)
df['num_claims']=df['num_claims'].astype(float) #int does not handle NAN values, so using float instead
# df=df.set_index('id')

In [5]:
df.head()

,date,num_claims
id,,
10000000,2018,20.0
10000001,2018,12.0
10000002,2018,9.0
10000003,2018,18.0
10000004,2018,6.0


In [6]:
#outer join to citation received
#there are patents who received citations that are not in patent.csv.gz
#for example, the very ancient ones

In [7]:
df2 = dd.read_parquet(received, compression='gzip')
df=df.join(df2, how='outer')
df['cit_received']=df['cit_received'].fillna(0)

In [8]:
#i dont see any reason for patents making citations not to appear in patent.csv.gz
#so from this data on, the list of patents is stable
#the number of rows should not change (check)
df2 = dd.read_parquet(made, compression='gzip')
df=df.join(df2, how='outer')

In [9]:
df2 = dd.read_parquet(originality, compression='gzip')
df=df.join(df2, how='outer')

In [10]:
df2 = dd.read_parquet(generality, compression='gzip')
df=df.join(df2, how='outer')

In [11]:
df2 = dd.read_parquet(wipo, compression='gzip')
df=df.join(df2, how='outer')

In [12]:
df2 = dd.read_parquet(int_ext_cit, compression='gzip', dtype=object)
# df.set_index('index', inplace=True)
df=df.join(df2, how='outer')

In [13]:
df2 = dd.read_csv(centrality, dtype=object)
df2=df2.set_index('id')
df=df.join(df2, how='outer')

In [14]:
df=df.compute()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32836551 entries, 000004 to pctus9926473
Data columns (total 11 columns):
 #   Column          Dtype   
---  ------          -----   
 0   date            float64 
 1   num_claims      float64 
 2   cit_received    float64 
 3   cit_made        float64 
 4   originality     float64 
 5   generality      float64 
 6   wipo_field_id   category
 7   wipo_sector_id  category
 8   external        float64 
 9   far_external    float64 
 10  pagerank        object  
dtypes: category(2), float64(8), object(1)
memory usage: 2.5+ GB


In [16]:
df.sample(n=10)

,date,num_claims,cit_received,cit_made,originality,generality,wipo_field_id,wipo_sector_id,external,far_external,pagerank
7908344,2011.0,16.0,1.0,10.0,0.000000,0.000000,06,Electrical Eng,2.0,2.0,4.540885174371385e-08
5764238,1998.0,19.0,107.0,4.0,0.000000,0.037370,06,Electrical Eng,1.0,1.0,1.781771511223168e-07
7229462,2007.0,2.0,23.0,100.0,0.059767,0.235537,13,Instruments,12.0,12.0,4.940950077531114e-08
5176137,1993.0,20.0,101.0,25.0,0.000000,0.057641,13,Instruments,2.0,2.0,2.0588309739916396e-07
8590211,2013.0,17.0,8.0,19.0,0.000000,0.000000,35,Other fields,1.0,1.0,1.0216406374224693e-07
8916025,2014.0,14.0,2.0,14.0,0.260355,0.000000,23,Chemistry,0.0,0.0,4.081450003572667e-08
3252220,NaN,NaN,12.0,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,1.1211748691803353e-07
7488115,2009.0,19.0,10.0,15.0,0.124444,0.540000,09,Instruments,3.0,3.0,6.92345416457745e-08
4883307,1989.0,11.0,41.0,10.0,0.000000,0.000000,32,Mechanical Eng,1.0,1.0,3.5639561937839067e-07
4192559,1980.0,7.0,8.0,4.0,0.000000,0.375000,31,Mechanical Eng,0.0,1.0,5.327296362130927e-08


In [17]:
df.describe()

,date,num_claims,cit_received,cit_made,originality,generality,external,far_external
count,3.059001e+07,3.059001e+07,3.283655e+07,3.015226e+07,2.948322e+07,2.447864e+07,2.891620e+07,2.891620e+07
mean,2.006712e+03,1.694335e+01,1.554219e+01,2.836698e+01,2.220709e-01,1.960301e-01,1.497436e+00,2.077250e+00
std,9.913002e+00,1.278786e+01,4.633784e+01,8.284519e+01,2.271192e-01,2.313882e-01,4.521593e+00,5.299073e+00
min,1.976000e+03,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+03,9.000000e+00,1.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,2.009000e+03,1.600000e+01,4.000000e+00,1.200000e+01,1.652893e-01,0.000000e+00,1.000000e+00,1.000000e+00
75%,2.015000e+03,2.000000e+01,1.300000e+01,2.400000e+01,4.444444e-01,4.303503e-01,1.000000e+00,2.000000e+00
max,2.020000e+03,8.870000e+02,4.677000e+03,5.841000e+03,8.000000e-01,8.000000e-01,4.390000e+02,4.420000e+02


In [18]:
df.describe(include=[np.object]).transpose()

,count,unique,top,freq
pagerank,32576644,6353520,3.881098877432008e-08,7337568


In [19]:
df.to_parquet(dst, compression='gzip')

In [20]:
# In[32]:


# this code includes WIPO in the dataset
# as class is also part of non-cited patents, I'm not including in here.
# in any case, for some reason this merge could not be done by the method used in the previous
# there is a bug when using read_csv.gz using dtype and indexing. 
# it is a known bug in the community but i do not know why the previous join did not accused the issue
# the method below corrects the issue 

# names=['index', 'id', 'field_id']
# dtype={'id':object}
# usecols=['id', 'field_id']
# # data_read(wipo, names, usecols, dtype)
# wipo = pd.read_csv.gz(wipo, names=names, usecols=usecols, dtype=dtype, header=0)
# wipo['id']=wipo['id'].astype(str)
# wipo=wipo.set_index('id')
# wipo.info()

# dfs.append(wipo)